### Check GPU availability

In [1]:
!nvidia-smi

Wed Dec 26 15:20:52 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.145                Driver Version: 384.145                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   52C    P2    65W / 250W |   1011MiB / 11172MiB |     22%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 28%   42C    P8    18W / 250W |     10MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

### Import libraries

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import math
import numpy as np
import time
import json
import plotly
import logging
logging.getLogger().setLevel(logging.INFO)

from tqdm import tqdm
from idst_util import trivial
from idst_util import dstc2

from plotly.graph_objs import Scatter, Layout
from plotly.graph_objs.layout import Margin
plotly.offline.init_notebook_mode(connected = True)

[nltk_data] Downloading package punkt to /home/is/andrei-
[nltk_data]     cc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Check DSTC2 availability

In [3]:
trivial.print_idst()
dstc2.check()

INFO:root:+--------------------------------+
INFO:root:|         _ ____  ___________    |
INFO:root:|        (_) __ \/ ___/_  __/    |
INFO:root:|       / / / / /\__ \ / /       |
INFO:root:|      / / /_/ /___/ // /        |
INFO:root:|     /_/_____//____//_/         |
INFO:root:|                                |
INFO:root:+--------------------------------+
INFO:root:|Incremental Dialog State Tracker|
INFO:root:+--------------------------------+
INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Checker           |
INFO:root:+--------------------------------+
INFO:root:Looking for dstc2 directory in .
INFO:root:dstc2 was found!
INFO:root:Looking for dstc2_traindev directory in ./dstc2
INFO:root:dstc2_traindev was found!
INFO:root:Looking for dstc2_test directory in ./dstc2
INFO:root:dstc2_test was found!
INFO:root:Looking for dstc2_scripts directory in ./dstc2
INFO:root:dstc2_scripts was found!
INFO:root:Done!


### Retrieve data

In [4]:
raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_Y_test, \
ontology = dstc2.retrieve_raw_datasets(train_data_augmentation = False)

INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|       Dataset Retrieval        |
INFO:root:+--------------------------------+
INFO:root:Reading dstc2_train.flist, dstc2_dev.flist and ontology_dstc2.json
INFO:root:Asserted 1612 dialogs for dstc2_train.flist
INFO:root:Asserted 506 dialogs for dstc2_dev.flist
INFO:root:Extracting raw train features
100%|██████████| 1612/1612 [00:08<00:00, 183.48it/s]
INFO:root:Extracting raw dev features
100%|██████████| 506/506 [00:03<00:00, 167.45it/s]
INFO:root:Reading dstc2_test.flist
INFO:root:Asserted 1117 dialogs for dstc2_test.flist
INFO:root:Extracting raw test features
100%|██████████| 1117/1117 [00:07<00:00, 155.21it/s]


### Set device

In [5]:
logging.info("+--------------------------------+")
logging.info("|            Baseline            |")
logging.info("+--------------------------------+")

GPU_ID = 1
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
#DEVICE = "cpu"
if DEVICE == "cpu":
    logging.warning("Running on CPU")
else:
    logging.info("Running on GPU {}".format(GPU_ID))

INFO:root:+--------------------------------+
INFO:root:|            Baseline            |
INFO:root:+--------------------------------+
INFO:root:Running on GPU 1


### Create vocabularies

In [6]:
logging.info("+--------------------------------+")
logging.info("|          Vocabulary            |")
logging.info("+--------------------------------+")
logging.info("Creating token_to_index, index_to_token and token_to_count dictionaries")

token_to_index = {"<unk>": 0}
index_to_token = {0: "<unk>"}
token_to_count = {"<unk>": 1}

for raw_train_dialog in tqdm(raw_X_train):
    
    for raw_train_turn in raw_train_dialog["turns"]:
        
        tokens_scores = raw_train_turn["system"] + raw_train_turn["user"]
        
        for token_score in tokens_scores:
            token = token_score[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
                
assert len(token_to_index) == len(index_to_token)
assert len(token_to_index) == len(token_to_count)

INFO:root:+--------------------------------+
INFO:root:|          Vocabulary            |
INFO:root:+--------------------------------+
INFO:root:Creating token_to_index, index_to_token and token_to_count dictionaries
100%|██████████| 1612/1612 [00:00<00:00, 19344.57it/s]


### Execution configuration

In [26]:
logging.info("+--------------------------------+")
logging.info("|         Configuration          |")
logging.info("+--------------------------------+")

VOCABULARY_SIZE = len(token_to_index)
EMBEDDING_DIM = 170
ALTERED_EMBEDDING_DIM = 300
HIDDEN_DIM = 100
NUM_EPOCHS = 100
GOAL_LOSS_FUNCTION = nn.CrossEntropyLoss()
METHOD_LOSS_FUNCTION = nn.CrossEntropyLoss()
REQUESTED_LOSS_FUNCTION = nn.BCELoss()

METHOD_DIM = len(ontology["method"])
REQUESTED_DIM = int(math.pow(2, len(ontology["requestable"])))
GOAL_FOOD_DIM = len(ontology["informable"]["food"]) + 2 # because of null and dontcare
GOAL_PRICERANGE_DIM = len(ontology["informable"]["pricerange"]) + 2 # because of null and dontcare
GOAL_NAME_DIM = len(ontology["informable"]["name"]) + 2 # because of null and dontcare
GOAL_AREA_DIM = len(ontology["informable"]["area"]) + 2 # because of null and dontcare

logging.info("VOCABULARY_SIZE:\t\t{}".format(VOCABULARY_SIZE))
logging.info("EMBEDDING_DIM:\t\t{}".format(EMBEDDING_DIM))
logging.info("ALTERED_EMBEDDING_DIM:\t{}".format(ALTERED_EMBEDDING_DIM))
logging.info("HIDDEN_DIM:\t\t\t{}".format(HIDDEN_DIM))
logging.info("NUM_EPOCHS:\t\t\t{}".format(NUM_EPOCHS))
logging.info("METHOD_DIM:\t\t\t{}".format(METHOD_DIM))
logging.info("REQUESTED_DIM:\t\t{}".format(REQUESTED_DIM))
logging.info("GOAL_FOOD_DIM:\t\t{}".format(GOAL_FOOD_DIM))
logging.info("GOAL_PRICERANGE_DIM:\t\t{}".format(GOAL_PRICERANGE_DIM))
logging.info("GOAL_NAME_DIM:\t\t{}".format(GOAL_NAME_DIM))
logging.info("GOAL_AREA_DIM:\t\t{}".format(GOAL_AREA_DIM))
logging.info("GOAL_LOSS_FUNCTION:\t\t{}".format(GOAL_LOSS_FUNCTION))
logging.info("METHOD_LOSS_FUNCTION:\t\t{}".format(METHOD_LOSS_FUNCTION))
logging.info("REQUESTED_LOSS_FUNCTION:\t\t{}".format(REQUESTED_LOSS_FUNCTION))

INFO:root:+--------------------------------+
INFO:root:|         Configuration          |
INFO:root:+--------------------------------+
INFO:root:VOCABULARY_SIZE:		923
INFO:root:EMBEDDING_DIM:		170
INFO:root:ALTERED_EMBEDDING_DIM:	300
INFO:root:HIDDEN_DIM:			100
INFO:root:NUM_EPOCHS:			100
INFO:root:METHOD_DIM:			5
INFO:root:REQUESTED_DIM:		256
INFO:root:GOAL_FOOD_DIM:		93
INFO:root:GOAL_PRICERANGE_DIM:		5
INFO:root:GOAL_NAME_DIM:		115
INFO:root:GOAL_AREA_DIM:		7
INFO:root:GOAL_LOSS_FUNCTION:		CrossEntropyLoss()
INFO:root:METHOD_LOSS_FUNCTION:		CrossEntropyLoss()
INFO:root:REQUESTED_LOSS_FUNCTION:		BCELoss()


### Models utilities

In [8]:
def get_index_and_score(turn, token_to_index, device):
    indices = []
    scores = []
    tokens_scores = turn["system"] + turn["user"]
    for token, score in tokens_scores:
        if token not in token_to_index:
            indices.append(token_to_index["<unk>"])
        else:
            indices.append(token_to_index[token])
        scores.append(score)
    assert len(indices) == len(scores)
    return torch.tensor(indices, dtype = torch.long, device = device), torch.tensor(scores, dtype = torch.float, device = device)

def plotly_plot(train_losses, dev_losses):
    plotly.offline.iplot({
                            "data": [Scatter(
                                            x = list(range(len(train_losses))),
                                            y = train_losses,
                                            mode = "lines+markers",
                                            name = "Train Loss",
                                            marker = dict(color = "#1abc9c")),
                                    Scatter(
                                            x = list(range(len(dev_losses))),
                                            y = dev_losses,
                                            mode = "lines+markers",
                                            name = "Dev Loss",
                                            marker = dict(color = "#3498db"))],
                            "layout": Layout(
                                             title = "<b>Train-Dev Loss</b>",
                                             xaxis = dict(title = "<b>Epoch</b>",
                                                          dtick = 1,
                                                          titlefont = dict(color = "#34495e")),
                                             yaxis = dict(title = "<b>Loss</b>",
                                                          titlefont = dict(color = "#34495e")),
                                             margin = Margin(b = 150))
                        })

class EarlyStopping():
    
    def __init__(self, min_delta = 0, patience = 1):
        
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf
        self.stop_training = False
    
    def on_epoch_end(self, epoch, current_loss):
        if np.less((current_loss - self.min_delta), self.best):
            self.best = current_loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait > self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
        return self.stop_training

get_bin = lambda x, n: format(x, "b").zfill(n)

def retrieve_output_Method(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    method_dict = {}
    
    index = torch.argmax(output_tensor).item()
    method_dict[ontology["method"][index]] = output_tensor[index].item()
    
    return method_dict

def retrieve_output_Requested(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    requested_dict = {}
    binary_representation = get_bin(torch.argmax(output_tensor).item(), len(ontology["requestable"]))
    indices = []
    index = 0
    for value in binary_representation:
        if int(value) == 1:
            indices.append(index)
        index += 1
    for index in indices:
        requested_dict[ontology["requestable"][index]] = (1 / len(indices))
    return requested_dict

def retrieve_output_GoalFood(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_food_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        if index == 1:
            goal_food_dict["dontcare"] = output_tensor[index].item() 
        else:
            goal_food_dict[ontology["informable"]["food"][index - 2]] = output_tensor[index].item()
        
    return goal_food_dict

def retrieve_output_GoalPricerange(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_pricerange_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        if index == 1:
            goal_pricerange_dict["dontcare"] = output_tensor[index].item()
        else:     
            goal_pricerange_dict[ontology["informable"]["pricerange"][index - 2]] = output_tensor[index].item()
        
    return goal_pricerange_dict

def retrieve_output_GoalName(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_name_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        if index == 1:
            goal_name_dict["dontcare"] = output_tensor[index].item()
        else:    
            goal_name_dict[ontology["informable"]["name"][index - 2]] = output_tensor[index].item()
        
    return goal_name_dict

def retrieve_output_GoalArea(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_area_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        if index == 1:
            goal_area_dict["dontcare"] = output_tensor[index].item()
        else:
            goal_area_dict[ontology["informable"]["area"][index - 2]] = output_tensor[index].item()
        
    return goal_area_dict

def make_tracker(raw_X, raw_Y, dataset):
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        for raw_X_dialog, raw_Y_dialog in tqdm(zip(raw_X, raw_Y), total = len(raw_X)):

            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for raw_X_turn, raw_Y_turn in zip(raw_X_dialog["turns"], raw_Y_dialog["turns"]):
                turn = {}
                turn["goal-labels"] = {}

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, device = DEVICE)
                
                output_MethodModel = model_Method(indices, scores)
                turn["method-label"] = retrieve_output_Method(output_MethodModel)

                output_RequestedModel = model_Requested(indices, scores)
                if torch.argmax(output_RequestedModel).item() == 0:
                    turn["requested-slots"] = {}
                else:
                    turn["requested-slots"] = retrieve_output_Requested(output_RequestedModel)

                goal_food, goal_pricerange, goal_name, goal_area = model_Goal(indices, scores)
                if torch.argmax(goal_food).item() != 0:
                    turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food)
                if torch.argmax(goal_pricerange).item() != 0:
                    turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange)
                if torch.argmax(goal_name).item() != 0:
                    turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name)
                if torch.argmax(goal_area).item() != 0:
                    turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area)

                session["turns"].append(turn)
            tracker_json["sessions"].append(session)
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        return tracker_json


### Goal

In [9]:
class GoalModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_food_dim,
                 goal_pricerange_dim,
                 goal_name_dim,
                 goal_area_dim,
                 device):
        
        super(GoalModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_food_dim = goal_food_dim
        self.goal_pricerange_dim = goal_pricerange_dim
        self.goal_name_dim = goal_name_dim
        self.goal_area_dim = goal_area_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_food_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_food_dim)
        
        self.goal_pricerange_classifier = nn.Linear(in_features = hidden_dim,
                                                    out_features = goal_pricerange_dim)
        
        self.goal_name_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_name_dim)
        
        self.goal_area_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_area_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_food = F.log_softmax(self.goal_food_classifier(self.hidden[0]).view(-1, self.goal_food_dim), dim = 1)        
        
        goal_pricerange = F.log_softmax(self.goal_pricerange_classifier(self.hidden[0]).view(-1, self.goal_pricerange_dim), dim = 1) 
        
        goal_name = F.log_softmax(self.goal_name_classifier(self.hidden[0]).view(-1, self.goal_name_dim), dim = 1)
        
        goal_area = F.log_softmax(self.goal_area_classifier(self.hidden[0]).view(-1, self.goal_area_dim), dim = 1)
        
        return goal_food, goal_pricerange, goal_name, goal_area

def retrieve_gold_GoalFood(raw_Y, ontology, device):
    ontology_informable_food = ontology["informable"]["food"]
    raw_goal_food = raw_Y["goal"]["food"]
    goal_food = 0
    if raw_goal_food != None:
        if raw_goal_food == "dontcare":
            goal_food = 1
        else:    
            goal_food = ontology_informable_food.index(raw_goal_food) + 2
    return torch.tensor([goal_food], dtype = torch.long, device = device)

def retrieve_gold_GoalPriceRange(raw_Y, ontology, device):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    raw_goal_pricerange = raw_Y["goal"]["pricerange"]
    goal_pricerange = 0
    if raw_goal_pricerange != None:
        if raw_goal_pricerange == "dontcare":
            goal_pricerange = 1
        else:    
            goal_pricerange = ontology_informable_pricerange.index(raw_goal_pricerange) + 2
    return torch.tensor([goal_pricerange], dtype = torch.long, device = device)

def retrieve_gold_GoalName(raw_Y, ontology, device):
    ontology_informable_name = ontology["informable"]["name"]
    raw_goal_name = raw_Y["goal"]["name"]
    goal_name = 0
    if raw_goal_name != None:
        if raw_goal_name == "dontcare":
            goal_name = 1
        else:    
            goal_name = ontology_informable_name.index(raw_goal_name) + 2
    return torch.tensor([goal_name], dtype = torch.long, device = device)

def retrieve_gold_GoalArea(raw_Y, ontology, device):
    ontology_informable_area = ontology["informable"]["area"]
    raw_goal_area = raw_Y["goal"]["area"]
    goal_area = 0
    if raw_goal_area != None:
        if raw_goal_area == "dontcare":
            goal_area = 1
        else:    
            goal_area = ontology_informable_area.index(raw_goal_area) + 2
    return torch.tensor([goal_area], dtype = torch.long, device = device)

model_Goal = GoalModel(vocabulary_size = VOCABULARY_SIZE,
                       embedding_dim = EMBEDDING_DIM,
                       altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                       hidden_dim = HIDDEN_DIM,
                       goal_food_dim = GOAL_FOOD_DIM,
                       goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                       goal_name_dim = GOAL_NAME_DIM,
                       goal_area_dim = GOAL_AREA_DIM,
                       device = DEVICE)

model_Goal = model_Goal.to(DEVICE)

optimizer_GoalModel = optim.Adam(model_Goal.parameters(), lr = 1e-4)

### Train Goal

In [10]:
train_losses = []
dev_losses = []
goal_early_stopping = EarlyStopping(patience = 3)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    train_loss = 0
    
    model_Goal = model_Goal.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm(zip(raw_X_train, raw_Y_train), total = len(raw_X_train)):

        model_Goal.hidden = model_Goal.init_hidden()

        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_GoalModel.zero_grad()
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, device = DEVICE)
            
            goal_food, goal_pricerange, goal_name, goal_area = model_Goal(indices, scores)
            
            loss_goal_food = LOSS_FUNCTION(goal_food,
                                           retrieve_gold_GoalFood(raw_Y_train_turn,
                                                                  ontology = ontology,
                                                                  device = DEVICE))
            
            loss_goal_pricerange = LOSS_FUNCTION(goal_pricerange,
                                                 retrieve_gold_GoalPriceRange(raw_Y_train_turn,
                                                                              ontology = ontology,
                                                                              device = DEVICE))
            
            loss_goal_name = LOSS_FUNCTION(goal_name,
                                           retrieve_gold_GoalName(raw_Y_train_turn,
                                                                  ontology = ontology,
                                                                  device = DEVICE))
            
            loss_goal_area = LOSS_FUNCTION(goal_area,
                                           retrieve_gold_GoalArea(raw_Y_train_turn,
                                                                  ontology = ontology,
                                                                  device = DEVICE))

            loss = loss_goal_food + loss_goal_pricerange + loss_goal_name + loss_goal_area
            loss.backward(retain_graph = True)
            
            train_loss += loss

            optimizer_GoalModel.step()
    
    train_losses.append(train_loss.item())
    
    model_Goal = model_Goal.eval()
    
    with torch.no_grad():
        
        dev_loss = 0
        
        for raw_X_dev_dialog, raw_Y_dev_dialog in tqdm(zip(raw_X_dev, raw_Y_dev), total = len(raw_X_dev)):
            
            for raw_X_dev_turn, raw_Y_dev_turn in zip(raw_X_dev_dialog["turns"], raw_Y_dev_dialog["turns"]):
                
                indices, scores = get_index_and_score(raw_X_dev_turn, token_to_index, device = DEVICE)
                
                goal_food, goal_pricerange, goal_name, goal_area = model_Goal(indices, scores)
            
                loss_goal_food = LOSS_FUNCTION(goal_food,
                                           retrieve_gold_GoalFood(raw_Y_dev_turn,
                                                                  ontology = ontology,
                                                                  device = DEVICE))
            
                loss_goal_pricerange = LOSS_FUNCTION(goal_pricerange,
                                                     retrieve_gold_GoalPriceRange(raw_Y_dev_turn,
                                                                                  ontology = ontology,
                                                                                  device = DEVICE))
            
                loss_goal_name = LOSS_FUNCTION(goal_name,
                                               retrieve_gold_GoalName(raw_Y_dev_turn,
                                                                      ontology = ontology,
                                                                      device = DEVICE))
            
                loss_goal_area = LOSS_FUNCTION(goal_area,
                                               retrieve_gold_GoalArea(raw_Y_dev_turn,
                                                                      ontology = ontology,
                                                                      device = DEVICE))

                loss = loss_goal_food + loss_goal_pricerange + loss_goal_name + loss_goal_area
                
                dev_loss += loss
                
        dev_losses.append(dev_loss.item())
    
    logging.info("Epoch train loss\t{} \tdev loss\t{}".format(train_loss, dev_loss))
    
    goal_early_stopping.on_epoch_end(epoch = (epoch + 1), current_loss = dev_loss.item())
    if goal_early_stopping.stop_training:
        break

plotly_plot(train_losses, dev_losses)

INFO:root:Epoch	1/100
100%|██████████| 506/506 [00:03<00:00, 131.84it/s]
INFO:root:Epoch train loss	62056.0859375 	dev loss	29265.01171875
INFO:root:Epoch	2/100
100%|██████████| 506/506 [00:03<00:00, 127.71it/s]
INFO:root:Epoch train loss	36326.12890625 	dev loss	31619.267578125
INFO:root:Epoch	3/100
100%|██████████| 506/506 [00:03<00:00, 129.00it/s]
INFO:root:Epoch train loss	26537.44140625 	dev loss	30221.65234375
INFO:root:Epoch	4/100
100%|██████████| 506/506 [00:04<00:00, 122.66it/s]
INFO:root:Epoch train loss	20566.419921875 	dev loss	31117.89453125
INFO:root:Epoch	5/100
100%|██████████| 506/506 [00:03<00:00, 130.81it/s]
INFO:root:Epoch train loss	16875.654296875 	dev loss	33185.671875


### Method

In [11]:
class MethodModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 method_dim,
                 device):
        
        super(MethodModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.method_dim = method_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        
        self.method_classifier = nn.Linear(in_features = hidden_dim,
                                           out_features = method_dim)
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
        
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        method = F.log_softmax(self.method_classifier(self.hidden[0]).view(-1, self.method_dim), dim = 1)
        
        return method

def retrieve_gold_Method(raw_Y, ontology, device):
    ontology_methods = ontology["method"]
    raw_goal_method = raw_Y["method"]
    goal_method = ontology_methods.index(raw_goal_method)
    return torch.tensor([goal_method], dtype = torch.long, device = device)

model_Method = MethodModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           method_dim = METHOD_DIM,
                           device = DEVICE)

model_Method = model_Method.to(DEVICE)

optimizer_MethodModel = optim.Adam(model_Method.parameters(), lr = 1e-4)

### Train Method

In [24]:
train_losses = []
dev_losses = []
method_early_stopping = EarlyStopping(patience = 3)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    train_loss = 0
    
    model_Method = model_Method.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm(zip(raw_X_train, raw_Y_train), total = len(raw_X_train)):

        model_Method.hidden = model_Method.init_hidden()

        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_MethodModel.zero_grad()
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, device = DEVICE)
            
            method = model_Method(indices, scores)
            
            loss_method = LOSS_FUNCTION(method,
                                        retrieve_gold_Method(raw_Y_train_turn,
                                                             ontology = ontology,
                                                             device = DEVICE))

            loss = loss_method
            loss.backward(retain_graph = True)
            
            train_loss += loss

            optimizer_MethodModel.step()
    
    train_losses.append(train_loss.item())
    
    model_Method = model_Method.eval()
    
    with torch.no_grad():
        
        dev_loss = 0
        
        for raw_X_dev_dialog, raw_Y_dev_dialog in tqdm(zip(raw_X_dev, raw_Y_dev), total = len(raw_X_dev)):
            
            for raw_X_dev_turn, raw_Y_dev_turn in zip(raw_X_dev_dialog["turns"], raw_Y_dev_dialog["turns"]):
                
                indices, scores = get_index_and_score(raw_X_dev_turn, token_to_index, device = DEVICE)
                
                method = model_Method(indices, scores)
            
                loss_method = LOSS_FUNCTION(method,
                                            retrieve_gold_Method(raw_Y_dev_turn,
                                                                 ontology = ontology,
                                                                 device = DEVICE))

                loss = loss_method
                
                dev_loss += loss
                
        dev_losses.append(dev_loss.item())
    
    logging.info("Epoch train loss\t{} \tdev loss\t{}".format(train_loss, dev_loss))
    
    method_early_stopping.on_epoch_end(epoch = (epoch + 1), current_loss = dev_loss.item())
    if method_early_stopping.stop_training:
        break

plotly_plot(train_losses, dev_losses)

INFO:root:Epoch	1/100
100%|██████████| 506/506 [00:02<00:00, 228.04it/s]
INFO:root:Epoch train loss	325.71685791015625 	dev loss	3554.88720703125
INFO:root:Epoch	2/100
100%|██████████| 506/506 [00:02<00:00, 214.45it/s]
INFO:root:Epoch train loss	287.8167419433594 	dev loss	3488.938232421875
INFO:root:Epoch	3/100
100%|██████████| 506/506 [00:02<00:00, 194.12it/s]
INFO:root:Epoch train loss	263.7943420410156 	dev loss	3869.174072265625
INFO:root:Epoch	4/100
100%|██████████| 506/506 [00:02<00:00, 226.66it/s]
INFO:root:Epoch train loss	220.06680297851562 	dev loss	3756.40380859375
INFO:root:Epoch	5/100
100%|██████████| 506/506 [00:02<00:00, 228.73it/s]
INFO:root:Epoch train loss	163.9022979736328 	dev loss	4092.485107421875
INFO:root:Epoch	6/100
100%|██████████| 506/506 [00:02<00:00, 228.39it/s]
INFO:root:Epoch train loss	212.81326293945312 	dev loss	4126.13134765625


### Requested

In [39]:
class RequestedModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 requested_dim,
                 device):
        
        super(RequestedModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.requested_dim = requested_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        
        self.requested_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = requested_dim)
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
        
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        requested = F.log_softmax(self.requested_classifier(self.hidden[0]).view(-1, self.requested_dim), dim = 1)
        
        return requested

def retrieve_gold_Requested(raw_Y, ontology, device):
    ontology_requestable = ontology["requestable"]
    raw_gold_requested = raw_Y["requested"]
    
    binary_representation = np.zeros(len(ontology_requestable), dtype = int)
    if len(raw_gold_requested) != 0:
        for requested in raw_gold_requested:
            binary_representation[ontology_requestable.index(requested)] = 1        
    
    goal_requested = binary_representation.dot(2**np.arange(binary_representation.size)[::-1])
    return torch.tensor([goal_requested], dtype = torch.long, device = device)

model_Requested = RequestedModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 requested_dim = REQUESTED_DIM,
                                 device = DEVICE)

model_Requested = model_Requested.to(DEVICE)

optimizer_RequestedModel = optim.Adam(model_Requested.parameters(), lr = 1e-4)

### Train requested

In [38]:
train_losses = []
dev_losses = []
requested_early_stopping = EarlyStopping(patience = 3)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    train_loss = 0
    
    model_Requested = model_Requested.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm(zip(raw_X_train, raw_Y_train), total = len(raw_X_train)):

        model_Requested.hidden = model_Requested.init_hidden()

        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_RequestedModel.zero_grad()
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, device = DEVICE)
            
            requested = model_Requested(indices, scores)
            
            loss_requested = LOSS_FUNCTION(requested,
                                           retrieve_gold_Requested(raw_Y_train_turn,
                                                                   ontology = ontology,
                                                                   device = DEVICE))

            loss = loss_requested
            loss.backward(retain_graph = True)
            
            train_loss += loss

            optimizer_RequestedModel.step()
    
    train_losses.append(train_loss.item())
    
    model_Requested = model_Requested.eval()
    
    with torch.no_grad():
        
        dev_loss = 0
        
        for raw_X_dev_dialog, raw_Y_dev_dialog in tqdm(zip(raw_X_dev, raw_Y_dev), total = len(raw_X_dev)):
            
            for raw_X_dev_turn, raw_Y_dev_turn in zip(raw_X_dev_dialog["turns"], raw_Y_dev_dialog["turns"]):
                
                indices, scores = get_index_and_score(raw_X_dev_turn, token_to_index, device = DEVICE)
                
                requested = model_Requested(indices, scores)
            
                loss_requested = LOSS_FUNCTION(requested,
                                               retrieve_gold_Requested(raw_Y_train_turn,
                                                                       ontology = ontology,
                                                                       device = DEVICE))

                loss = loss_requested
                
                dev_loss += loss
                
        dev_losses.append(dev_loss.item())
    
    logging.info("Epoch train loss\t{} \tdev loss\t{}".format(train_loss, dev_loss))
    
    requested_early_stopping.on_epoch_end(epoch = (epoch + 1), current_loss = dev_loss.item())
    if requested_early_stopping.stop_training:
        break

plotly_plot(train_losses, dev_losses)

INFO:root:Epoch	1/100
  0%|          | 0/1612 [00:00<?, ?it/s]

tensor([[-5.6140, -5.4299, -5.4088, -5.6420, -5.3438, -5.5471, -5.6019, -5.4331,
         -5.6174, -5.4986, -5.6191, -5.5933, -5.5579, -5.6108, -5.5441, -5.6242,
         -5.5358, -5.5078, -5.4885, -5.5132, -5.4639, -5.4281, -5.4012, -5.5116,
         -5.5585, -5.5213, -5.6575, -5.6388, -5.5333, -5.5615, -5.6193, -5.5859,
         -5.6142, -5.6450, -5.5609, -5.5972, -5.5624, -5.4820, -5.5974, -5.5909,
         -5.5328, -5.6893, -5.5049, -5.5123, -5.5010, -5.6114, -5.4485, -5.4432,
         -5.5600, -5.6439, -5.4342, -5.4170, -5.5154, -5.7173, -5.6305, -5.5002,
         -5.4812, -5.4953, -5.6523, -5.5927, -5.4863, -5.5492, -5.5415, -5.5643,
         -5.5128, -5.6093, -5.4060, -5.4821, -5.5698, -5.5678, -5.6648, -5.6202,
         -5.5427, -5.4937, -5.5196, -5.4598, -5.6063, -5.5115, -5.5104, -5.6541,
         -5.7142, -5.4950, -5.5828, -5.4984, -5.5499, -5.5100, -5.5303, -5.4734,
         -5.6288, -5.5345, -5.5487, -5.5131, -5.6933, -5.6442, -5.6862, -5.5009,
         -5.6951, -5.5749, -

ValueError: Expected input batch_size (1) to match target batch_size (8).

In [ ]:
dev_tracker = make_tracker(raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

with open("dev_tracker.json", "w") as dev_tracker_file:
    json.dump(dev_tracker, dev_tracker_file)

!python2 dstc2/dstc2_scripts/score.py\
--dataset dstc2_dev\
--dataroot dstc2/dstc2_traindev/data\
--ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
--trackfile dev_tracker.json\
--scorefile dev_tracker.score.csv

!python2 dstc2/dstc2_scripts/report.py --scorefile dev_tracker.score.csv